In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import shapely
from shapely.geometry import MultiPolygon,Polygon
from tueplots import bundles
import contextily as cx
from src.PlotFuncitons import *
from src.DataFunctions import *
from src.DataLoaders import LKS01

%matplotlib inline
%config InlineBackend.figure_format = 'retina' # does not affect other users
# or consider setting your dpi in rcParams to a reasonably high value (also important for exporting if not explicitly given)

'''
# add lightness to colormap
lightness = .8
my_cmap = plt.cm.Oranges(np.arange(plt.cm.Oranges.N))
my_cmap[:,0:3] *= lightness
my_cmap[:,0:3] += (1-lightness)
my_cmap = np.flip(my_cmap,0)
my_cmap = ListedColormap(my_cmap)
'''
my_cmap = 'Oranges'
ax_labels = {'ylabel':"Northing (meter)",
          'xlabel':"Easting (meter)"}

In [ ]:
plz_shape_df = gpd.read_file('Datasets/PLZ/plz-5stellig/plz-5stellig.shp', dtype={'plz': str}).to_crs(epsg=3857)
plz_shape_df.drop(['einwohner','qkm'],axis=1,inplace=True)
plz_shape_df.head(3)

In [ ]:
plz_region_df = pd.read_csv('Datasets/PLZ/zuordnung_plz_ort.csv',sep=',',dtype={'plz': str})
plz_region_df.drop(['osm_id','ags','landkreis'], axis=1, inplace=True)
plz_region_df.head(3)

In [ ]:
# Merge data.
germany_df = pd.merge(
    left=plz_shape_df, 
    right=plz_region_df, 
    on='plz',
    how='inner'
)
germany_df.drop(['note'], axis=1, inplace=True)
germany_df.head(3)

In [ ]:
plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()

germany_df.plot(
    ax=ax, 
    column='bundesland', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'Bundesland','bbox_to_anchor': (1.5,.8)},
    cmap='tab20',
    alpha=.9
)
plot_cities(ax)
ax.set(title='Germany - Federal States',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

In [ ]:
plz_einwohner_df = pd.read_csv(
    'Datasets/PLZ/plz_einwohner.csv', 
    sep=',', 
    dtype={'plz': str, 'einwohner': int}
)
plz_einwohner_df.drop(['note','qkm','lat','lon'],axis=1,inplace=True)
# Merge geo and inhabitant data
germany_df = pd.merge(
    left=germany_df, 
    right=plz_einwohner_df, 
    on='plz',
    how='left'
)
germany_df.head(3)

## Plot continuous distribution of inhabitants

In [ ]:
plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
germany_df.plot(
    ax=ax, 
    column='einwohner', 
    categorical=False, 
    legend=True, 
    cmap=my_cmap
    )
plot_cities(ax)
ax.set(title='Germany: Number of Inhabitants per Postal Code',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

## Plot discrete distribution in line with PKS "Tatortverteilung"

In [ ]:
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
germany_df['einwohner_kategorie'] = pd.cut(germany_df['einwohner'], bins=bins, labels=labels, right=False)

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
germany_df.plot(
    ax=ax, 
    column='einwohner_kategorie', 
    legend=True, 
    cmap=my_cmap
)
plot_cities(ax)  
ax.set(title='Germany: Number of Inhabitants per Postal Code',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

Not the distribution we are looking for. Larger cities should have >500k inhabitants. Let's try grouping by postal codes instead.

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/plz-5stellig/plz-5stellig.shp', dtype={'plz': str}).to_crs(epsg=3857)
plz_map = pd.read_csv('Datasets/PLZ/georef-germany-postleitzahl.csv',sep=';',dtype={'Postleitzahl / Post code':str})
plz_map = plz_map.rename(columns={'Postleitzahl / Post code':'plz'})
merged_df = pd.merge(geo_df, plz_map, on='plz',how='left')
merged_df.head(3)

In [ ]:
grouped_df = merged_df.groupby('PLZ Name (short)').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')

grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap=my_cmap)
plot_cities(ax)
ax.set(title='Germany: Number of Inhabitants per Postal Code',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

Looking better, but we are missing quite a lot of data. Let's switch to a another geodataset.

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/georef-germany-postleitzahl/georef-germany-postleitzahl.shp').to_crs(epsg=3857)
geo_df.head(3)

In [ ]:
plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig,ax = plt.subplots()
geo_df.plot(
    ax=ax, 
    column='lan_code', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'First Digit', 'bbox_to_anchor': (1.25,.8)},
    cmap='tab20',
    alpha=.9
)
plot_cities(ax)
ax.set(title='Germany',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

Darker regions show some overlap in the regions, but overall not too bad. Merge it with inhabitants on postal codes.

In [ ]:
inhabitans_df = pd.read_csv('Datasets/PLZ/plz_einwohner.csv',dtype={'plz':str}).rename(columns={'plz':'plz_code'})
merged_df = pd.merge(geo_df,inhabitans_df,on='plz_code',how='left')
merged_df.head(3)

Try grouping inhabitants by "Kreis" codes this time.

In [ ]:
grouped_df = merged_df.groupby('krs_code').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap=my_cmap)
plot_cities(ax)
ax.set(title="Germany: Number of Inhabitants per 'Kreis' Code",**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

Granularity is too low this time as we are not seeing any regions with <20k inhabitants. Try postal code names as last resort...

In [ ]:
grouped_df = merged_df.groupby('plz_name').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x]),
    'einwohner': 'sum'
}).reset_index()
grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry='geometry')
bins = [0, 20000, 100000, 500000, float('inf')]
labels = ['0-20k', '20k-100k', '100k-500k', '> 500k']
grouped_gdf['einwohner_kategorie'] = pd.cut(grouped_gdf['einwohner'], bins=bins, labels=labels, right=False)

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
grouped_gdf.plot(ax=ax,column='einwohner_kategorie', legend=True,cmap=my_cmap)
plot_cities(ax)
ax.set(title="Germany: Number of Inhabitants per Postal Code Name",**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

Looks reasonable. However, we can verify that this is not the exact binning used by the BKA by looking at the data from Berlin:

Here we have all the cases in the >500k category and none in the lower bins, but our plot still splits Berlin into many smaller parts with <500k inhabitants. While this is the most obvious issue, we have no way automatically checking for more problems as there is no detailed explanation in the PKS tables as to what exactly is considered a "region". *

* BIK regions might be possible, however that data is not publicly available (perhaps on request).

## Rest....


Load crime data

clean existing geo data

reduce to federal states since we have spatial distribution of crimes for that as well.

## Reduce GeoData to Federal States

Problems:
- merging GeoDataFrame on federal states leaves noisy borders (solve by shrinking and expanding with buffer; only slight loss of precision)
- federal states in this dataset are not free of overlaps and show up in plots (solve by manually checking maps and removing overlap from the state that is too large)

In [ ]:
geo_df = gpd.read_file('Datasets/PLZ/georef-germany-postleitzahl/georef-germany-postleitzahl.shp')
geo_df.rename(columns={'lan_name':'Bundesland'},inplace=True)
bu_geo_full = geo_df.groupby('Bundesland').agg({
    'geometry': lambda x: MultiPolygon([geom if isinstance(geom, Polygon) else geom.geoms for geom in x])
}).reset_index()


# reduce geo information to outline of states
bu_geo_reduced = bu_geo_full.copy()
bu_geo_reduced['geometry'] = bu_geo_full['geometry'].apply(lambda x: shapely.ops.unary_union(x))

# re-compute shapes with slight buffering to avoid holes where borders don't line up perfectly
bu_geo_smooth = bu_geo_reduced.copy()
bu_geo_smooth['geometry'] = bu_geo_reduced['geometry'].apply(
    lambda x: x.buffer(1e-5, 1, join_style=shapely.geometry.JOIN_STYLE.mitre).buffer(-1e-4, 1, join_style=shapely.geometry.JOIN_STYLE.mitre))

bu_geo_full = gpd.GeoDataFrame(bu_geo_full,geometry='geometry',crs='EPSG:4326').to_crs(epsg=3857)
bu_geo_reduced = gpd.GeoDataFrame(bu_geo_reduced,geometry='geometry',crs='EPSG:4326').to_crs(epsg=3857)
bu_geo_smooth = gpd.GeoDataFrame(bu_geo_smooth,geometry='geometry',crs='EPSG:4326').to_crs(epsg=3857)

overlaps = {'BU_1':[],'BU_2':[],'geometry':[]}
for _,bu1 in bu_geo_smooth.iterrows(): 
    for _,bu2 in bu_geo_smooth.iterrows():
        if bu1['Bundesland'] != bu2['Bundesland']:
            overlaps['BU_1'].append(bu1['Bundesland'])
            overlaps['BU_2'].append(bu2['Bundesland'])
            overlaps['geometry'].append(bu1.geometry.intersection(bu2.geometry))
overlaps = gpd.GeoDataFrame(overlaps,geometry='geometry')
overlaps = overlaps[~overlaps['geometry'].is_empty].reset_index(drop=True) # drop empty intersections

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig,axs = plt.subplots(1,3,sharex=True,sharey=True,layout='tight')
bu_geo_full.plot(ax=axs[0],alpha=.5,edgecolor='black',linewidth=.5)
axs[0].set(title='All regions')
bu_geo_reduced.plot(ax=axs[1],alpha=.5,edgecolor='black',linewidth=.5)
axs[1].set(title='Grouped by federal state')
bu_geo_smooth.plot(ax=axs[2],alpha=.5,edgecolor='black',linewidth=.5)
overlaps.plot(ax=axs[2],facecolor='red',edgecolor='none')
axs[2].set(title='Overlapping regions after smoothing')
fig.supxlabel('Easting (meter)')
fig.supylabel('Northing (meter)')
plt.show()

In [ ]:
geo = bu_geo_smooth.copy()

def remove_intersection(region1, region2):
    '''Removes the intersecting region from region 1

    Input:
    :param: region1: The region which is too large
    :param: region2: The region which the area actally belongs to
    
    Output:
    :return: Reduced shape of region 1
    '''
    reduced = []
    for pol1 in region1:
        for pol2 in region2:
            if pol1.intersects(pol2):
                # If they intersect, create a new polygon that is
                # essentially pol1 minus the intersection
                nonoverlap = (pol1.symmetric_difference(pol2)).difference(pol2)
                reduced.append(list(nonoverlap.geoms) if isinstance(nonoverlap,MultiPolygon) else nonoverlap)
            else:
                # Otherwise, just keep the initial polygon as it is.
                reduced.append(pol1)
    return shapely.ops.unary_union(reduced)


def cleanBU_(df,bu1,bu2):
    df.loc[df.loc[:,"Bundesland"] == bu1,"geometry"] = remove_intersection(
        df.loc[df.loc[:,"Bundesland"] == bu1,"geometry"],
        df.loc[df.loc[:,"Bundesland"] == bu2,"geometry"])


cleanBU_(geo,"Hessen","Niedersachsen")
cleanBU_(geo,"Hessen","Rheinland-Pfalz")
cleanBU_(geo,"Hessen","Nordrhein-Westfalen")
cleanBU_(geo,"Bayern","Baden-Württemberg")
cleanBU_(geo,"Baden-Württemberg","Hessen")
cleanBU_(geo,"Thüringen","Sachsen")
cleanBU_(geo,"Schleswig-Holstein","Hamburg")
cleanBU_(geo,"Mecklenburg-Vorpommern","Niedersachsen")
cleanBU_(geo,"Mecklenburg-Vorpommern","Brandenburg")
cleanBU_(geo,"Sachsen-Anhalt","Brandenburg")

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
ax = geo.plot(alpha=0.7,edgecolor='black',linewidth=.5,facecolor='orange')
ax.set(title='Overlap-free geo data',**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

## Add population data

In [ ]:
# source: https://www.destatis.de/DE/Themen/Laender-Regionen/Regionales/Gemeindeverzeichnis/Administrativ/02-bundeslaender.html
population = pd.read_excel('Datasets/Destatis/02-bundeslaender.xlsx', sheet_name='Bundesländer_mit_Hauptstädten',skiprows=6,skipfooter=16)
population.drop(columns=['Unnamed: 1','Unnamed: 4', 'Unnamed: 5'],inplace=True)
population.rename(columns={'Unnamed: 0':'Bundesland',
                           'Unnamed: 2':'Fläche',
                           'Unnamed: 3':'Einwohner',
                           'Unnamed: 6':'Dichte'},
                           inplace=True)
idx = [i for i in range(len(population)) if i % 2 != 0]
population = population.drop(idx)
population.Bundesland.replace(r'\d+', '',regex=True,inplace=True)
population.Bundesland = population.Bundesland.str.strip()
population = population.sort_values('Bundesland').reset_index(drop=True)
population.at[16,'Bundesland'] = 'Deutschland'
population.head(3)

In [ ]:
data = LKS01()
cases = data[2022].loc[data[2022]['Schlüssel'] == '------']
merged_df = add_geomery(cases,geo)

plt.rcParams.update(bundles.icml2022(column='full',usetex=False))
fig, ax = plt.subplots()
merged_df.plot(ax=ax,
               column='Anzahl erfasste Fälle',
               legend=True,
               cmap=my_cmap,
               legend_kwds={'label':'Absolute number of crimes'},
               edgecolor='black',
               linewidth=.1)
plot_cities(ax)
ax.set(title="Germany 2022: Crimes per federal state",**ax_labels)
cx.add_basemap(ax, source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
plt.show()

## Cybercrime per federal state over time

In [ ]:
merged_data = []
# no "%-Anteil an allen Fällen" column available before 2019; compute it manually:
print('Loading 2013 - 2022 ...')
for year in range(2013,2019):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    df_year_all = df_year[df_year['Schlüssel'] == '------']['Anzahl erfasste Fälle'].to_numpy()
    df_year = df_year.loc[df_year['Schlüssel'] == '897000']
    df_year['%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    merged_year = pd.merge(df_year,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))
for year in range(2019,2023):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    df_year = df_year.loc[df_year['Schlüssel'] == '897000']
    merged_year = pd.merge(df_year,geo,on='Bundesland')
    merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))

In [ ]:
vmin = min([x['%-Anteil an allen Fällen'].min() for x in merged_data])
vmax = max([x['%-Anteil an allen Fällen'].max() for x in merged_data])

fig, axs = plt.subplots(3,4,layout='constrained',figsize=(8,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        merged_data[i].plot(ax=ax,
                            column='%-Anteil an allen Fällen',
                            legend=False,
                            cmap=my_cmap,
                            vmin=vmin,
                            vmax=vmax,
                            edgecolor='black',
                            linewidth=.1)
        ax.set(title=i+2013)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,my_cmap,'% of all crime')
fig.suptitle("Germany: Cybercrime per federal state (relative)")
plt.show()

Huh? Shouldn't 2020 and 2021 be overall higher than other years? (see temporal analysis)

Problem: For some reason the fraction of all crime in 2020 and 2021 is not computed wrt. the number of crimes in each federal state, but in all of Germany.

In [ ]:
def add_fraction(df_year):
    df_year_all = df_year[df_year['Schlüssel'] == '------']['Anzahl erfasste Fälle'].to_numpy()
    df_year = df_year.loc[df_year['Schlüssel'] == '897000'].copy()
    if '%-Anteil an allen Fällen' in df_year.columns:
        df_year.loc[:,'%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    else:
       df_year['%-Anteil an allen Fällen'] = df_year['Anzahl erfasste Fälle'].to_numpy() / df_year_all * 100
    return df_year

merged_data = []
# no "%-Anteil an allen Fällen" column available before 2019; compute it manually and fix 2020,2021:
print('Loading 2013 - 2022 ...')
for year in range(2013,2023):
    print(f'Year: {year}', end="\r", flush=True)
    df_year = data[year]
    if year in [2019,2022]:
        df_year = df_year.loc[df_year['Schlüssel'] == '897000']
        merged_year = pd.merge(df_year,geo,on='Bundesland')
        merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))
    else:
        df_year = add_fraction(df_year)
        merged_year = pd.merge(df_year,geo,on='Bundesland')
        merged_data.append(gpd.GeoDataFrame(merged_year, geometry='geometry'))

In [ ]:
vmin = min([x['%-Anteil an allen Fällen'].min() for x in merged_data])
vmax = max([x['%-Anteil an allen Fällen'].max() for x in merged_data])

fig, axs = plt.subplots(3,4,layout='constrained',figsize=(8,7))
for i,ax in enumerate(axs.flatten()):
    if i < len(merged_data):
        im = merged_data[i].plot(ax=ax,
                            column='%-Anteil an allen Fällen',
                            legend=False,
                            cmap=my_cmap,
                            vmin=vmin,
                            vmax=vmax,
                            edgecolor='black',
                            linewidth=.1)
        ax.set(title=i+2013)
    ax.set_axis_off()
    if i == len(merged_data):
        plot_cbar(ax,vmin,vmax,my_cmap,'% of all crime')
fig.suptitle("Germany: Cybercrime per federal state (relative)")
plt.show()

## Create one massive plot with everything related to cybercrime

Categories:
- absolute
- % of all crime
- HZ
- absolute / density

In [ ]:
density = population.Dichte.to_numpy()
inhabitants = population.Einwohner.to_numpy()


# absolute already exists
data_2016 = get_key_col(add_fraction(data[2016]),'897000')
# HZ nach Zensus already exists
data_2016['Nach Dichte']= data_2016['Anzahl erfasste Fälle'].to_numpy() / density

data_2016_geo = add_geomery(data_2016,geo)




data_2022 = get_key_col(data[2022],'897000')

data_2022['HZ nach Zensus']= data_2022['Anzahl erfasste Fälle'].to_numpy() / inhabitants * 1e5
data_2022['Nach Dichte']= data_2022['Anzahl erfasste Fälle'].to_numpy() / density
data_2022_geo = add_geomery(data_2022,geo)

In [ ]:
plt.rcParams.update(bundles.icml2022(column='full',usetex=False,nrows=4,ncols=4))
fig, axs = plt.subplots(3,4,gridspec_kw={'height_ratios': [1,1,.05]})

def get_kws(col_name):
    return {'column':col_name,
            'legend':False,
            'cmap':my_cmap,
            'vmin':get_min(col_name),
            'vmax':get_max(col_name),
            'edgecolor':'black',
            'linewidth':.1,
            'rasterized':True}

def get_min(col_name):
    return min(data_2016_geo[col_name].min(),data_2022_geo[col_name].min())

def get_max(col_name):
    return max(data_2016_geo[col_name].max(),data_2022_geo[col_name].max())

def plot_cbar(ax,vmin,vmax,cmap,label=None,layout:str='vertical'):
    if layout == 'vertical':
        gradient = np.linspace(vmax, vmin, 256)
        gradient = np.vstack((gradient, gradient)).T
        pos1 = ax.get_position() # get the original position 
        pos2 = [pos1.x0 - (pos1.width / 2), pos1.y0,  pos1.width / 1, pos1.height / 1.5] 
        ax.set_position(pos2) # set a new position
        ax.imshow(gradient,cmap=cmap,aspect=.15)
        ax.set_axis_on()
        ax.yaxis.tick_right()
        ax.set_yticks(range(10,247,59)) # 5 ticks
        ax.set_yticklabels([f'{x:.2g}' for x in np.linspace(vmax*1.039, vmin*0.961, 5)]) # scale to match 10/256 and 246/256
        ax.set_xticks([])
        if label:
            ax.set_ylabel(label)
            ax.yaxis.set_label_position("right")
    if layout == 'horizontal':
        gradient = np.linspace(vmax, vmin, 256)
        gradient = np.vstack((gradient, gradient))
        ax.imshow(gradient,cmap=cmap,aspect=10)
        ax.set_axis_on()
        ax.set_xticks(range(10,247,59*2)) # 3 ticks
        ax.set_xticklabels([f'{x:.3g}' for x in np.linspace(vmax*1.039, vmin*0.961, 3)],rotation=0) # scale to match 10/256 and 246/256
        ax.set_yticks([])
        if label:
            ax.set_xlabel(label)


categories = ['Anzahl erfasste Fälle',
             '%-Anteil an allen Fällen',
             'HZ nach Zensus',
             'Nach Dichte']

labels = ['# cases',
          '% cybercrime',
          '# cases / # inhabitants',
          '# cases / density']

for i,category in enumerate(categories):
    data_2016_geo.plot(ax=axs[0,i],**get_kws(category))
    data_2022_geo.plot(ax=axs[1,i],**get_kws(category))
    axs[0,i].set(xticks=[],yticks=[])
    axs[1,i].set(xticks=[],yticks=[])
    plot_cbar(axs[2,i],get_min(category),get_max(category),my_cmap,labels[i],layout='horizontal')
    cx.add_basemap(axs[0,i], source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
    cx.add_basemap(axs[1,i], source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
axs[0,0].set(ylabel='2016')
axs[1,0].set(ylabel='2022')

fig.suptitle("Germany: Dsitribution of Cybercrime")
plt.rcParams['savefig.dpi'] = 500
plt.savefig('doc/icml2023/fig/spatial_analysis.pdf')
plt.show()

same plot but rotated by 90 degree

In [ ]:
plt.rcParams.update(bundles.icml2022(column='half',usetex=False,nrows=11,ncols=3))
fig, axs = plt.subplots(4,3,gridspec_kw={'width_ratios': [1,1,.05]})


def plot_cbar(ax,vmin,vmax,cmap,label=None,layout:str='vertical'):
    if layout == 'vertical':
        gradient = np.linspace(vmax, vmin, 256)
        gradient = np.vstack((gradient, gradient)).T
        #pos1 = ax.get_position() # get the original position 
        #pos2 = [pos1.x0 - (pos1.width / 2), pos1.y0,  pos1.width / 1, pos1.height / 1.5] 
        #ax.set_position(pos2) # set a new position
        ax.imshow(gradient,cmap=cmap,aspect=.2)
        ax.set_axis_on()
        ax.yaxis.tick_right()
        ax.set_yticks(range(10,247,59)) # 5 ticks
        ax.set_yticklabels([f'{x:.2g}' for x in np.linspace(vmax*1.039, vmin*0.961, 5)]) # scale to match 10/256 and 246/256
        ax.set_xticks([])
        if label:
            ax.set_ylabel(label)
            ax.yaxis.set_label_position("right")
    if layout == 'horizontal':
        gradient = np.linspace(vmax, vmin, 256)
        gradient = np.vstack((gradient, gradient))
        ax.imshow(gradient,cmap=cmap,aspect=10)
        ax.set_axis_on()
        ax.set_xticks(range(10,247,59*2)) # 3 ticks
        ax.set_xticklabels([f'{x:.3g}' for x in np.linspace(vmax*1.039, vmin*0.961, 3)],rotation=0) # scale to match 10/256 and 246/256
        ax.set_yticks([])
        if label:
            ax.set_xlabel(label)


categories = ['Anzahl erfasste Fälle',
             '%-Anteil an allen Fällen',
             'HZ nach Zensus',
             'Nach Dichte']

labels = ['# cases',
          '% cybercrime',
          '# cases / # inhabitants',
          '# cases / density']

for i,category in enumerate(categories):
    data_2016_geo.plot(ax=axs[i,0],**get_kws(category))
    data_2022_geo.plot(ax=axs[i,1],**get_kws(category))
    axs[i,0].set(xticks=[],yticks=[],ylabel=labels[i])
    axs[i,1].set(xticks=[],yticks=[])
    plot_cbar(axs[i,2],get_min(category),get_max(category),my_cmap,layout='vertical') # labels on cbar instead of y axis
    cx.add_basemap(axs[i,0], source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
    cx.add_basemap(axs[i,1], source=cx.providers.Esri.WorldPhysical , zoom=6, attribution=False)
axs[0,0].set(title='2016')
axs[0,1].set(title='2022')

fig.suptitle("Germany: Dsitribution of Cybercrime")
plt.rcParams['savefig.dpi'] = 500
plt.savefig('doc/icml2023/fig/spatial_analysis_v.pdf')
plt.show()